In [266]:
import json
import pandas as pd
from time import sleep
import asyncio
from solana.rpc.async_api import AsyncClient
from datetime import datetime
from itertools import chain
from pydantic import BaseModel, Json
import requests
from typing import Any

### Marketplace addrs:
#### Thanks Levi for this gist: https://gist.github.com/levicook/34f390073bd57abebc786cda5bec4094

In [249]:
marketplace_mapper = {
    "HZaWndaNWHFDd9Dhk5pqUUtsmoBCqzb1MLu3NAh1VX6B": "AlphaArt",
    "A7p8451ktDCHq5yYaHczeLMYsjRsAkzc3hCXcSrwYHU7": "DigitalEyes",
    "AmK5g2XcyptVLCFESBCJqoSfwV3znGoVYQnqEnaAZKWn": "ExchangeArt",
    "MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8": "MagicEden",
    "CJsLwbP1iu5DuUikHEJnLfANgKy6stB2uFgvBBHoyxwz": "Solanart"
}

marketplace_token_url_mapper = {
    "AlphaArt": "https://alpha.art/t/",
    "DigitalEyes": "https://digitaleyes.market/item/",
    "ExchangeArt": "https://exchange.art/single/",
    "MagicEden": "https://magiceden.io/item-details/",
    "Solanart": "https://solanart.io/search/" # params: ?token=<>
}

marketplace_fetch_id_url = "https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress"

### Setting up http client using genesysgo rpc

In [61]:
SSC_RPC_ENDPOINT = "https://ssc-dao.genesysgo.net"
SOL_EXPLORER_RPC_ENDPOINT = "https://explorer-api.mainnet-beta.solana.com"

In [73]:
sol_client = AsyncClient(SSC_RPC_ENDPOINT)

### Fetching data for all the accounts

In [67]:
SSC_METADATA_JSON_URL = "https://sld-gengo.s3.amazonaws.com"

In [68]:
SSC_ADDR = "D6wZ5U9onMC578mrKMp5PZtfyc5262426qKsYJW7nT3p"

In [69]:
ssc_account_data = await sol_client.get_account_info(SSC_ADDR)

In [70]:
ssc_account_data

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 112104841},
  'value': {'data': ['', 'base64'],
   'executable': False,
   'lamports': 957628287585,
   'owner': '11111111111111111111111111111111',
   'rentEpoch': 259}},
 'id': 1}

In [18]:
total_txns = await sol_client.get_transaction_count()

In [19]:
total_txns

{'jsonrpc': '2.0', 'result': 45865897950, 'id': 5}

In [74]:
ssc_signatures = await sol_client.get_confirmed_signature_for_address2(SSC_ADDR)

In [75]:
len(ssc_signatures["result"])

763

In [76]:
ssc_signatures["result"][-1]

{'blockTime': 1639272701,
 'confirmationStatus': 'finalized',
 'err': None,
 'memo': None,
 'signature': '3WbxAAnGHgsdzLz2BnN1EULSMWKxfM64SSsTvnVdfgFMYVPFVWp7WvF5i3957sjSCrxW1588AL74F2mv5y6yeTE1',
 'slot': 111140025}

In [77]:
ssc_signatures_prev1 = await sol_client.get_signatures_for_address(
    SSC_ADDR, 
    before="3WbxAAnGHgsdzLz2BnN1EULSMWKxfM64SSsTvnVdfgFMYVPFVWp7WvF5i3957sjSCrxW1588AL74F2mv5y6yeTE1", limit=5)

In [120]:
def fetch_time_diff(ts_start: datetime, ts_end: datetime, time_format: str = "seconds"):
    return round((ts_end - ts_start).total_seconds(), 2)

async def fetch_txn_batch(rpc_endpoint: str, batch_count: int, cursor_addr: str = None):
    async with AsyncClient(rpc_endpoint) as sol_client:
        ssc_signatures = await sol_client.get_confirmed_signature_for_address2(
            SSC_ADDR,
            limit=batch_count,
            before=cursor_addr
        )
        result = []
        if len(ssc_signatures["result"]) != 0:
            result = ssc_signatures["result"]
        return result

async def fetch_all_txns_for_ssc(rpc_endpoint: str, batch_count: int = 500) -> pd.DataFrame:
    ts_start = datetime.now()
    txn_list = []
    cursor_addr = None
    while True:
        txns = await fetch_txn_batch(rpc_endpoint=rpc_endpoint, 
                                               batch_count=batch_count,
                                               cursor_addr=cursor_addr)
        if len(txns) == 0:
            ts_end = datetime.now()
            break
        cursor_addr = txns[-1]["signature"]
        print(f"Last txn: {cursor_addr} | Len: {len(txns)}")
        txn_list.append(txns)
    final_txn_list = list(chain.from_iterable(txn_list))
    print(f"{len(final_txn_list)} signatures have been fetched in {fetch_time_diff(ts_start=ts_start, ts_end=ts_end)} secs")
    return final_txn_list

In [121]:
results = await fetch_all_txns_for_ssc(rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

Last txn: 23VRvtu1QZcP37RdSPYXuzLRZkeNprGwjSFgrzRGeiXQmP6upS5iTRc6vdSRG5sNU7nsFEbnZ5RaQ9ARUuUAF6An | Len: 500
Last txn: Y3ubH5W2voTnwjZg3dtcEVvrPX4fu7p6vHcW7cwwX9Zd6Z1xEmfRYU39PmyhhVmFqyxfXe5CX4w94pnLx9BLKHC | Len: 500
Last txn: 3kb32L5HMHpxjhYTXZB3vPgPN1dR8sarWxxmx8FZS8T4cVVe6pByxmdiuMSBbmhX4WHLzePnsjeKu7kfWgLoN46x | Len: 500
Last txn: cskiTmfS3mn2ZcgSHEryHzqiTY28CEkf2HguEmKZCQzCRHvnMt8m3iJBxNsd14hLG2xS6qQRZYVkM1Qoa3jgfRG | Len: 500
Last txn: 3BeZC1APLk4bDxFbLCeDhQiUKayHwpb9xWDdq3yQsJnnbV9yCi4zbKu558nK8JMqyeqafPjKExBbV7ceVbvgFKQH | Len: 500
Last txn: 5sM11d8e5HPRJ3TGPYJNhmCSYC47f2V9YGPnmuMYz7wWiycRXPK2F9AajFrPyiom29bvtU7ehFHAZap7nnDy5EHp | Len: 500
Last txn: 2zUsc78VfsNt7itN36YERx4XUEZxk8hhRfqjJHdMLT47N7VxfRo6UtkzrUcZe16LdKPAfmh7iSGtY5ALjXeKASw | Len: 500
Last txn: 3PDNDhQvqTwt29nRc9GpXYhdZuF8psxKsH8fuUCvikEaw8jfZc9UDaS2bDMJWnp2SHeP3p2jFhKmcMmDpFhdK75q | Len: 500
Last txn: 5UuMxxfaRPrWyRsDSi3CZwHuhEq7HJjUBFhdn5d56fyrYX4YdopcSVu448XzUAxgZsz37WHHkR6nJXXusuSxm81Q | Len: 431
4431 signatur

### Fetching txn details

In [123]:
results[0]

{'blockTime': 1639820827,
 'confirmationStatus': 'finalized',
 'err': None,
 'memo': None,
 'signature': '5VnTzUTY1xE2TmiUiJiRFMV9tXMbH7LA7QKPpo79UY2EP2QPw9fbpifCEHfV8LBRiwNwE7fnbMkyvrHhgLLy3T2o',
 'slot': 112104216}

#### 1 SOL = 1_000_000_000 lamports

In [189]:
def convert_lamport_to_sol(amount: float):
    return amount / 1_000_000_000

In [164]:
async def fetch_raw_txn_details(txn: str, rpc_endpoint: str=SSC_RPC_ENDPOINT):
    async with AsyncClient(rpc_endpoint) as sol_client:
        return await sol_client.get_transaction(txn)

In [165]:
deets = await fetch_raw_txn_details(results[850]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

In [183]:
post = deets["result"]["meta"]["postBalances"]

In [184]:
pre = deets["result"]["meta"]["preBalances"]

In [190]:
[convert_lamport_to_sol(bals[0] - bals[1]) for bals in list(zip(pre, post))]

[28.000005,
 0.0,
 -26.04144768,
 0.00144768,
 -0.56,
 0.0,
 -1.4,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [267]:
class SscNftMetadata(BaseModel):
    seller: str
    purchaser: str
    selling_price: float
    seller_cut: float
    marketplace: str
    marketplace_contract_addr: str
    nft_addr: str
    nft_token_id: str # eg. Token id == SSC#4009
    nft_image_url: str
    nft_traits: Any
    nft_marketplace_url: str

In [268]:
async def fetch_token_info(nft_addr: str):
    resp = requests.get(f"{marketplace_fetch_id_url}/{nft_addr}")
    print(resp.url)
    if resp.status_code != 200:
        return None
    nft_metadata = resp.json()["results"]
    return nft_metadata["img"], nft_metadata["attributes"], nft_metadata["title"], nft_metadata["content"] 

In [277]:
async def fetch_txn_details(txn: str, rpc_endpoint: str=SSC_RPC_ENDPOINT):
    formatted_details = {}
    async with AsyncClient(rpc_endpoint) as sol_client:
        txn_details = await sol_client.get_transaction(txn)
        nft_addr = txn_details["result"]["meta"]["postTokenBalances"][0]["mint"]
        pre_balances = txn_details["result"]["meta"]["preBalances"]
        post_balances = txn_details["result"]["meta"]["postBalances"]
        actual_balances = [convert_lamport_to_sol(bals[0] - bals[1]) for bals in list(zip(pre_balances, post_balances))]
        addrs = txn_details["result"]["transaction"]["message"]["accountKeys"]
        balance_list = list(zip(addrs, actual_balances))
        nft_metadata = await fetch_token_info(nft_addr=nft_addr)
        marketplace_name = marketplace_mapper[balance_list[-1][0]]
        metadata = SscNftMetadata(
            seller=balance_list[2][0],
            purchaser=balance_list[0][0],
            selling_price=balance_list[0][1],
            seller_cut=-1 * (balance_list[2][1]),
            marketplace=marketplace_name,
            marketplace_contract_addr=balance_list[-1][0],
            nft_addr=nft_addr,
            nft_token_id=nft_metadata[2],
            nft_image_url=nft_metadata[0],
            nft_traits=nft_metadata[1],
            nft_marketplace_url=f"{marketplace_token_url_mapper[marketplace_name]}{nft_addr}"
        )
        return metadata.dict()

In [278]:
await fetch_txn_details(results[850]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/AgQhbkA9zG5ri4fRVrbQXCKETTKz5g3kEa6JUjuNshwi


{'seller': '7PwyTH7PuCmndcUYhNL1hB22TVTdsss2Vof89zsaPoSy',
 'purchaser': 'kXsgQVop8DkL75AfPG8Zrx7WyoYaKcZpC3L9hePzVXq',
 'selling_price': 28.000005,
 'seller_cut': 26.04144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': 'AgQhbkA9zG5ri4fRVrbQXCKETTKz5g3kEa6JUjuNshwi',
 'nft_token_id': 'Shadowy Super Coder #231',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/231.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Green'},
  {'trait_type': 'Base Model', 'value': 'Tan'},
  {'trait_type': 'Desk', 'value': 'Red Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Steven'},
  {'trait_type': 'Monitor 2', 'value': 'Fabricpunk'},
  {'trait_type': 'Monitor 3', 'value': 'Linux'},
  {'trait_type': 'Mouth', 'value': 'Red Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Punk Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'C#'}],
 'nft_marketplace_url': 'https://magiceden.io/item-details/AgQhbkA

In [279]:
await fetch_txn_details(results[100]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/5QFF2Xh7Dnn8dzBFLyAZ3Ytrp2jdBDEqPn277FH98HVf


{'seller': 'ETf9WGPVpCZm4rAUvMx64EGpxLy1kjbdXhdCcg8fLYD5',
 'purchaser': '2UH3RjR4dDw6GAMZAE4bXF2VqBHjATSV6MuJqXH288ch',
 'selling_price': 46.300005,
 'seller_cut': 43.06044768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '5QFF2Xh7Dnn8dzBFLyAZ3Ytrp2jdBDEqPn277FH98HVf',
 'nft_token_id': 'Shadowy Super Coder #7167',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/7167.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Green'},
  {'trait_type': 'Base Model', 'value': 'Chestnut'},
  {'trait_type': 'Desk', 'value': 'Red Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Trading Dark'},
  {'trait_type': 'Monitor 2', 'value': 'Glitch Two'},
  {'trait_type': 'Monitor 3', 'value': 'Hacked'},
  {'trait_type': 'Mouth', 'value': 'Purple Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Unmasked Squid Game Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Go'}],
 'nft_marketplace_url': 'https://

In [280]:
await fetch_txn_details(results[4000]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/38tpc8NhdaTjPu1Xa9qpYXpYnYpTjLFPM6VsY7rFKU8i


{'seller': 'GeRDkBTfLBPj8dN9r5DJYT4Xv9eayLhMUhEQ83EDocyE',
 'purchaser': 'GQk1FGpgAC64vfxqZWjWBCreTT4ZRNg3aLGv6JxU2muJ',
 'selling_price': 8.500005,
 'seller_cut': 7.90644768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '38tpc8NhdaTjPu1Xa9qpYXpYnYpTjLFPM6VsY7rFKU8i',
 'nft_token_id': 'Shadowy Super Coder #7133',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/7133.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Pink'},
  {'trait_type': 'Base Model', 'value': 'Pale'},
  {'trait_type': 'Desk', 'value': 'Bronze Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Linux'},
  {'trait_type': 'Monitor 2', 'value': 'Glitch Two'},
  {'trait_type': 'Monitor 3', 'value': 'Star Atlas'},
  {'trait_type': 'Mouth', 'value': 'Purple Smile'},
  {'trait_type': 'Hoodie', 'value': 'Monke Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Python'}],
 'nft_marketplace_url': 'https://magiceden.io/item

In [281]:
await fetch_txn_details(results[200]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/FXuP8jDdxWSSFc6AS7kf9hkE1fp6BaNThNx2CwahWQ7P


{'seller': '97N4Ktgqy1rcGBmpJigjy8wZk9mECtZ86GdboeXYUq8V',
 'purchaser': '8QRR52guaXe46roHE8tW7KQ62GRRnbfxXmhdKBRmVtwu',
 'selling_price': 35.000005,
 'seller_cut': 32.55144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': 'FXuP8jDdxWSSFc6AS7kf9hkE1fp6BaNThNx2CwahWQ7P',
 'nft_token_id': 'Shadowy Super Coder #8453',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/8453.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Light-Orange'},
  {'trait_type': 'Base Model', 'value': 'Tan'},
  {'trait_type': 'Desk', 'value': 'Silver Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Linux'},
  {'trait_type': 'Monitor 2', 'value': 'Access Granted'},
  {'trait_type': 'Monitor 3', 'value': 'Scoopshop'},
  {'trait_type': 'Mouth', 'value': 'Red Smile'},
  {'trait_type': 'Hoodie', 'value': 'Squid Game Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'R'}],
 'nft_marketplace_url': 'https://magicede

In [282]:
await fetch_txn_details(results[10]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/85TGW42Q96CqAKwMrTY8Zq9nuAbRQVJdkvSQfWxK1FGp


{'seller': 'HH2yy6E9iBAXqUsw65i1YWBUo1akdjPsfft5wGU51su2',
 'purchaser': '78rZtPQwnvojcpKh1VJfUEvKi9BC6hFRMxjq5jv8qpEf',
 'selling_price': 44.000005,
 'seller_cut': 40.92144768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '85TGW42Q96CqAKwMrTY8Zq9nuAbRQVJdkvSQfWxK1FGp',
 'nft_token_id': 'Shadowy Super Coder #6347',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/6347.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Light-Blue'},
  {'trait_type': 'Base Model', 'value': 'Pale'},
  {'trait_type': 'Desk', 'value': 'Silver Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Trading White'},
  {'trait_type': 'Monitor 2', 'value': 'Hacked'},
  {'trait_type': 'Monitor 3', 'value': 'Ninja'},
  {'trait_type': 'Mouth', 'value': 'Pink Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Punk Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'C'}],
 'nft_marketplace_url': 'https://magiceden.io/item-

In [283]:
await fetch_txn_details(results[0]["signature"], rpc_endpoint=EXPLORER_MAINNET_RPC_ENDPOINT)

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/3m9BreHL2FUFgnK5FHngr1wVf5kshVre1EMtbSjPZ1Cz


{'seller': 'EUozZKRKemnTk3PaGGXE18TCs8uJZZagxbdCL5U4hWF6',
 'purchaser': '4REnZop7oP5yJcxUevLwUqGWsGf2SA6MY8tdQfTaQbNk',
 'selling_price': 44.500005,
 'seller_cut': 41.38644768,
 'marketplace': 'MagicEden',
 'marketplace_contract_addr': 'MEisE1HzehtrDpAAT8PnLHjpSSkRYakotTuJRPjTpo8',
 'nft_addr': '3m9BreHL2FUFgnK5FHngr1wVf5kshVre1EMtbSjPZ1Cz',
 'nft_token_id': 'Shadowy Super Coder #280',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/280.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Dark-Red'},
  {'trait_type': 'Base Model', 'value': 'Pale'},
  {'trait_type': 'Desk', 'value': 'Purple Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Trading Dark'},
  {'trait_type': 'Monitor 2', 'value': 'Matrix'},
  {'trait_type': 'Monitor 3', 'value': 'Glitch Two'},
  {'trait_type': 'Mouth', 'value': 'Red Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Sci-Fi Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'C++'}],
 'nft_marketplace_url': 'https://magiceden.io/it

In [284]:
await fetch_txn_details("5yZNYqNi13NDTzW5BdDHucJpeG6SumL1y4G3AwfBiTgVdJpb1d12RmaisSM7yi3r1nHuaLLmQQHvLwkSrzr3Z5TG")

https://api-mainnet.magiceden.io/rpc/getNFTByMintAddress/CJGqBx5GJT6JwVcz5V437MTC1bsMjNHfJCv7XQUA2n7F


{'seller': 'FaoTWCDskkJH6iro4kDrugWzE61smKhXqFJhDWoGBzVf',
 'purchaser': 'SgpSEypfA5M9h4Pnx7KUFR5xCQgosi7gBAkodCsqrYY',
 'selling_price': 5e-06,
 'seller_cut': -0.0,
 'marketplace': 'Solanart',
 'marketplace_contract_addr': 'CJsLwbP1iu5DuUikHEJnLfANgKy6stB2uFgvBBHoyxwz',
 'nft_addr': 'CJGqBx5GJT6JwVcz5V437MTC1bsMjNHfJCv7XQUA2n7F',
 'nft_token_id': 'Shadowy Super Coder #8654',
 'nft_image_url': 'https://sld-gengo.s3.amazonaws.com/8654.png',
 'nft_traits': [{'trait_type': 'Background', 'value': 'Noire'},
  {'trait_type': 'Base Model', 'value': 'Chestnut'},
  {'trait_type': 'Desk', 'value': 'Red Metal'},
  {'trait_type': 'Monitor 1', 'value': 'Matrix'},
  {'trait_type': 'Monitor 2', 'value': 'Samo'},
  {'trait_type': 'Monitor 3', 'value': 'Levi Korg'},
  {'trait_type': 'Mouth', 'value': 'Purple Smirk'},
  {'trait_type': 'Hoodie', 'value': 'Tech Hacker'},
  {'trait_type': 'Favorite Programming Language', 'value': 'Solidity'}],
 'nft_marketplace_url': 'https://solanart.io/search/CJGqBx5GJT6